#### Imports

In [1]:
import pandas as pd
import numpy as np

from sklearn.model_selection import cross_val_score, train_test_split, RepeatedStratifiedKFold
from sklearn.model_selection import GridSearchCV, RandomizedSearchCV
from sklearn.preprocessing import MinMaxScaler, RobustScaler, StandardScaler

from sklearn.linear_model import LinearRegression
from sklearn.multioutput import MultiOutputRegressor

- Data Load

In [2]:
test = pd.read_csv('test.csv')
test

,ID,X_01,X_02,X_03,X_04,X_05,X_06,X_07,X_08,X_09,...,X_47,X_48,X_49,X_50,X_51,X_52,X_53,X_54,X_55,X_56
0,TEST_00001,68.504,103.321,76.67,1,101.867,73.963,30.51,63.57,239.80,...,1,1,17227.63,138.130429,129.460682,141.506570,133.427229,129.711498,133.138096,121.859684
1,TEST_00002,67.485,103.320,69.37,1,101.992,67.845,28.03,116.99,189.23,...,1,1,17134.53,136.148839,128.266277,145.911745,131.196417,132.411480,133.629025,124.178623
2,TEST_00003,69.524,103.320,68.97,1,101.884,77.022,29.65,205.68,214.93,...,1,1,14860.83,120.447446,119.988804,132.099908,120.450155,130.051708,128.252972,114.475628
3,TEST_00004,69.524,103.320,65.87,1,101.866,73.963,28.15,103.38,180.80,...,1,1,15252.53,133.994695,125.069180,147.507669,123.142653,125.963665,139.666592,126.589253
4,TEST_00005,73.603,103.321,66.67,1,101.891,74.983,29.92,71.20,231.93,...,1,1,10752.23,137.918202,135.116192,138.600473,127.173033,137.252712,134.411335,124.020016
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
39603,TEST_39604,68.504,103.320,63.97,1,103.157,68.864,29.49,116.35,284.16,...,1,1,62123.53,127.741246,126.494312,139.119905,125.271109,128.284572,140.176945,128.292843
39604,TEST_39605,68.504,103.320,61.37,1,103.137,68.864,32.29,116.28,272.41,...,1,1,61844.13,127.767377,124.062809,138.238664,119.879393,127.322529,137.312047,131.570614
39605,TEST_39606,69.524,103.320,63.67,1,103.149,69.884,30.00,113.05,295.54,...,1,1,60277.53,128.593640,124.774037,138.659624,123.999571,126.075542,135.656132,127.671108
39606,TEST_39607,67.485,103.321,61.77,1,103.148,67.845,32.05,115.05,267.26,...,1,1,60236.73,121.110646,125.471699,134.989984,120.889578,129.296909,132.673977,131.882893


In [3]:
train = pd.read_csv('train.csv').filter(regex='X') # Input : X Featrue
target = pd.read_csv('train.csv').filter(regex='Y') # Output : Y Feature

In [4]:
train

,X_01,X_02,X_03,X_04,X_05,X_06,X_07,X_08,X_09,X_10,...,X_47,X_48,X_49,X_50,X_51,X_52,X_53,X_54,X_55,X_56
0,70.544,103.320,67.47,1,101.892,74.983,29.45,62.38,245.71,0.0,...,1,1,9706.03,137.043591,135.359219,147.837968,134.313475,125.605427,136.721425,125.028256
1,69.524,103.321,65.17,1,101.944,72.943,28.73,61.23,233.61,0.0,...,1,1,10423.43,133.736691,135.979817,149.924692,123.630583,127.893337,143.322659,124.877308
2,72.583,103.320,64.07,1,103.153,72.943,28.81,105.77,272.20,0.0,...,1,1,10948.53,132.805112,131.055355,146.814592,128.939070,127.012195,140.395688,122.238232
3,71.563,103.320,67.57,1,101.971,77.022,28.92,115.21,255.36,0.0,...,1,1,15007.03,134.138760,133.239422,139.720132,132.260824,130.723186,147.624829,134.875225
4,69.524,103.320,63.57,1,101.981,70.904,29.68,103.38,241.46,0.0,...,1,1,11051.03,142.728970,136.620022,134.853555,134.760252,125.647793,139.331105,123.272762
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
39602,66.465,103.320,62.27,1,103.150,66.825,30.20,77.83,298.05,0.0,...,1,1,60630.73,129.965741,130.807148,133.481737,125.273130,121.780933,133.780110,129.029812
39603,66.465,103.321,62.77,1,102.021,66.825,29.21,102.25,270.67,0.0,...,1,1,60763.43,127.633885,120.158764,142.667802,122.465490,122.987209,143.090741,122.811413
39604,68.504,103.320,64.67,1,103.144,68.864,29.96,102.61,198.07,0.0,...,1,1,8813.33,132.501286,136.893025,134.419328,129.115431,130.920147,140.489232,119.166699
39605,66.465,103.320,63.67,1,102.025,67.845,30.30,112.60,275.52,0.0,...,1,1,62222.33,128.189679,121.495930,141.288011,130.141676,125.518825,136.603634,124.525929


In [5]:
target

,Y_01,Y_02,Y_03,Y_04,Y_05,Y_06,Y_07,Y_08,Y_09,Y_10,Y_11,Y_12,Y_13,Y_14
0,2.056,1.456,1.680,10.502,29.632,16.083,4.276,-25.381,-25.529,-22.769,23.792,-25.470,-25.409,-25.304
1,1.446,1.184,1.268,18.507,33.179,16.736,3.229,-26.619,-26.523,-22.574,24.691,-26.253,-26.497,-26.438
2,1.251,0.665,0.782,14.082,31.801,17.080,2.839,-26.238,-26.216,-22.169,24.649,-26.285,-26.215,-26.370
3,1.464,1.079,1.052,16.975,34.503,17.143,3.144,-25.426,-25.079,-21.765,24.913,-25.254,-25.021,-25.345
4,0.983,0.646,0.689,15.047,32.602,17.569,3.138,-25.376,-25.242,-21.072,25.299,-25.072,-25.195,-24.974
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
39602,1.382,1.215,1.263,10.874,29.194,16.582,3.410,-26.486,-26.581,-22.772,24.261,-26.491,-26.584,-26.580
39603,1.482,0.606,1.083,8.759,29.859,15.659,3.406,-27.308,-27.203,-24.674,23.427,-27.250,-27.334,-27.325
39604,1.117,1.154,0.993,13.159,24.720,16.823,3.215,-26.502,-26.687,-22.577,24.301,-26.388,-26.425,-26.601
39605,0.895,0.187,0.477,9.123,26.412,15.757,4.216,-26.760,-26.634,-24.066,23.305,-26.536,-26.751,-26.635


In [6]:
x_info = pd.read_csv('x_feature_info.csv')
x_info

,Feature,설명
0,X_01,PCB 체결 시 단계별 누름량(Step 1)
1,X_02,PCB 체결 시 단계별 누름량(Step 2)
2,X_03,방열 재료 1 무게
3,X_04,1차 검사 통과 여부
4,X_05,PCB 체결 시 단계별 누름량(Step 3)
5,X_06,PCB 체결 시 단계별 누름량(Step 4)
6,X_07,방열 재료 1 면적
7,X_08,방열 재료 2 면적
8,X_09,방열 재료 3 면적
9,X_10,방열 재료 2 무게


In [7]:
y_info = pd.read_csv('y_feature_info.csv')
y_info

,Feature,설명
0,Y_01,안테나 Gain 평균 (각도1)
1,Y_02,안테나 1 Gain 편차
2,Y_03,안테나 2 Gain 편차
3,Y_04,평균 신호대 잡음비
4,Y_05,안테나 Gain 평균 (각도2)
5,Y_06,신호대 잡음비 (각도1)
6,Y_07,안테나 Gain 평균 (각도3)
7,Y_08,신호대 잡음비 (각도2)
8,Y_09,신호대 잡음비 (각도3)
9,Y_10,신호대 잡음비 (각도4)


In [8]:
y_spec = pd.read_csv('y_feature_spec_info.csv')
y_spec

,Feature,최소,최대
0,Y_01,0.2,2.0
1,Y_02,0.2,2.1
2,Y_03,0.2,2.1
3,Y_04,7.0,19.0
4,Y_05,22.0,36.5
5,Y_06,-19.2,19.0
6,Y_07,2.4,4.0
7,Y_08,-29.2,-24.0
8,Y_09,-29.2,-24.0
9,Y_10,-30.6,-20.0


---

In [9]:
train.columns=x_info.설명
train

설명,PCB 체결 시 단계별 누름량(Step 1),PCB 체결 시 단계별 누름량(Step 2),방열 재료 1 무게,1차 검사 통과 여부,PCB 체결 시 단계별 누름량(Step 3),PCB 체결 시 단계별 누름량(Step 4),방열 재료 1 면적,방열 재료 2 면적,방열 재료 3 면적,방열 재료 2 무게,...,3차 검사 통과 여부,4차 검사 통과 여부,Cal 투입 전 대기 시간,RF1 부분 SMT 납 량,RF2 부분 SMT 납 량,RF3 부분 SMT 납 량,RF4 부분 SMT 납 량,RF5 부분 SMT 납 량,RF6 부분 SMT 납 량,RF7 부분 SMT 납 량
0,70.544,103.320,67.47,1,101.892,74.983,29.45,62.38,245.71,0.0,...,1,1,9706.03,137.043591,135.359219,147.837968,134.313475,125.605427,136.721425,125.028256
1,69.524,103.321,65.17,1,101.944,72.943,28.73,61.23,233.61,0.0,...,1,1,10423.43,133.736691,135.979817,149.924692,123.630583,127.893337,143.322659,124.877308
2,72.583,103.320,64.07,1,103.153,72.943,28.81,105.77,272.20,0.0,...,1,1,10948.53,132.805112,131.055355,146.814592,128.939070,127.012195,140.395688,122.238232
3,71.563,103.320,67.57,1,101.971,77.022,28.92,115.21,255.36,0.0,...,1,1,15007.03,134.138760,133.239422,139.720132,132.260824,130.723186,147.624829,134.875225
4,69.524,103.320,63.57,1,101.981,70.904,29.68,103.38,241.46,0.0,...,1,1,11051.03,142.728970,136.620022,134.853555,134.760252,125.647793,139.331105,123.272762
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
39602,66.465,103.320,62.27,1,103.150,66.825,30.20,77.83,298.05,0.0,...,1,1,60630.73,129.965741,130.807148,133.481737,125.273130,121.780933,133.780110,129.029812
39603,66.465,103.321,62.77,1,102.021,66.825,29.21,102.25,270.67,0.0,...,1,1,60763.43,127.633885,120.158764,142.667802,122.465490,122.987209,143.090741,122.811413
39604,68.504,103.320,64.67,1,103.144,68.864,29.96,102.61,198.07,0.0,...,1,1,8813.33,132.501286,136.893025,134.419328,129.115431,130.920147,140.489232,119.166699
39605,66.465,103.320,63.67,1,102.025,67.845,30.30,112.60,275.52,0.0,...,1,1,62222.33,128.189679,121.495930,141.288011,130.141676,125.518825,136.603634,124.525929


In [10]:
X_train, X_test, y_train, y_test = train_test_split(train, target, test_size=0.2, random_state=2020)
X_train.shape, X_test.shape, y_train.shape, y_test.shape

((31685, 56), (7922, 56), (31685, 14), (7922, 14))

In [11]:
import sklearn.metrics as metrics

In [12]:
# iloc 추가

def lg_nrmse(gt, preds):
    # 각 Y Feature별 NRMSE 총합
    # Y_01 ~ Y_08 까지 20% 가중치 부여
    all_nrmse = []
    for idx in range(14): # ignore 'ID'
        rmse = metrics.mean_squared_error(gt.iloc[:,idx], preds.iloc[:,idx], squared=False)
        nrmse = rmse/np.mean(np.abs(gt.iloc[:,idx]))
        all_nrmse.append(nrmse)
    score = 1.2 * np.sum(all_nrmse[:8]) + 1.0 * np.sum(all_nrmse[8:15])
    return score

In [13]:
LR = MultiOutputRegressor(LinearRegression()).fit(X_train, y_train)
pred = pd.DataFrame(LR.predict(X_test))
lg_nrmse(y_test,pred)

2.0027532154755363

---

In [14]:
from sklearn.ensemble import RandomForestRegressor, GradientBoostingRegressor
from sklearn.tree import DecisionTreeRegressor
from sklearn.ensemble import ExtraTreesRegressor
from xgboost import XGBRegressor
from lightgbm import LGBMRegressor
from sklearn.linear_model import ElasticNet
from sklearn.ensemble import AdaBoostRegressor
from catboost import CatBoostRegressor, Pool
from ngboost import NGBRegressor
#from sklearn.ensemble import HistGradientBoostingRegressor
from sklearn.linear_model import LinearRegression, Lasso, Ridge, ElasticNet

In [150]:
#etr=ExtraTreesRegressor(n_estimators=200)
lr=LinearRegression()
#rg=Ridge()
#rf=RandomForestRegressor(random_state = 42, criterion = 'mae')
gbr=GradientBoostingRegressor(random_state = 42, max_depth = 4, learning_rate=0.09, loss='huber', n_estimators=130)
#xgb=XGBRegressor(random_state = 42, learning_rate=0.05, n_estimators=200)
lgb=LGBMRegressor(random_state = 42, learning_rate=0.05, n_estimators=200,
                 object='multiclass', metric='rmse')
#ada=AdaBoostRegressor(random_state = 42, learning_rate=0.03, n_estimators=100)
#hgb=HistGradientBoostingRegressor(random_state=42, scoring=nmae_score, learning_rate=0.09)
#ngb=NGBRegressor(random_state = 42, n_estimators = 500, verbose = 0, learning_rate = 0.03)
cat_reg=CatBoostRegressor(depth = 4, random_state = 42, loss_function = 'RMSE', n_estimators = 3000, learning_rate = 0.03, verbose = 0, task_type='GPU')

In [122]:
#etr.fit(X_train,y_train)
#lr.fit(X_train,y_train)
#rg.fit(X_train,y_train)
#rf.fit(X_train,y_train)
gbr = MultiOutputRegressor(gbr).fit(X_train,y_train)
#xgb.fit(X_train,y_train)
#lgb = MultiOutputRegressor(lgb).fit(X_train,y_train)
#ada.fit(X_train,y_train)
#hgb.fit(X_train,y_train)
#ngb.fit(X_train,y_train)
#cat_reg.fit(X_train,y_train)

In [123]:
#y_etr=pd.DataFrame(etr.predict(X_test))
#y_lr=pd.DataFrame(lr.predict(X_test))
#y_rg=pd.DataFrame(rg.predict(X_test))
#y_rf=pd.DataFrame(rf.predict(X_test))
y_gbr=pd.DataFrame(gbr.predict(X_test))
#y_xgb=pd.DataFrame(xgb.predict(X_test))
#y_lgb=pd.DataFrame(lgb.predict(X_test))
#y_ada=pd.DataFrame(ada.predict(X_test))
#y_hgb=pd.DataFrame(hgb.predict(X_test))
#y_ngb=pd.DataFrame(ngb.predict(X_test))
#y_cat=pd.DataFrame(cat_reg.predict(X_test))

In [124]:
#score_etr=lg_nrmse(y_test, y_etr)
#score_lr=lg_nrmse(y_test, y_lr)
#score_rg=lg_nrmse(y_test, y_rg)
#score_rf=lg_nrmse(y_test, y_rf)
score_gbr=lg_nrmse(y_test, y_gbr)
#score_xgb=lg_nrmse(y_test, y_xgb)
#score_lgb=lg_nrmse(y_test, y_lgb)
#score_ada=lg_nrmse(y_test, y_ada)
#score_hgb=lg_nrmse(y_test, y_hgb)
#score_ngb=lg_nrmse(y_test, y_ngb)
#score_cat=lg_nrmse(y_test, y_cat)

In [125]:
#print(score_etr)
#print(score_lr)
#print(score_rg)
#print(score_rf)
print(score_gbr)
#print(score_xgb)
#print(score_lgb)
#print(score_ada)
#print(score_hgb)
#print(score_ngb)
#print(score_cat)

1.985539139452176


In [181]:
import optuna

def objective(trial):

    param = {
        'objective': 'regression', # 회귀
        'verbose': -1,
        'metric': 'rmse', 
        'max_depth': trial.suggest_int('max_depth',3, 15),
        'learning_rate': trial.suggest_loguniform("learning_rate", 1e-8, 1e-2),
        'n_estimators': trial.suggest_int('n_estimators', 100, 3000),
        'min_child_samples': trial.suggest_int('min_child_samples', 5, 100),
        'subsample': trial.suggest_loguniform('subsample', 0.4, 1),
    }

    model = LGBMRegressor(**param)
    lgb_model = MultiOutputRegressor(model).fit(X_train, y_train)
    rmse = lg_nrmse(y_test, lgb_model.predict(X_test))
    return rmse
        
study_lgb = optuna.create_study(direction='minimize')
study_lgb.optimize(objective, n_trials=100)
print(study_lgb.best_trial)

[I 2022-08-19 16:59:34,084] A new study created in memory with name: no-name-b959fcf3-e431-40a6-b0b2-16209cdd5c6e
[W 2022-08-19 17:00:11,105] Trial 0 failed because of the following error: AttributeError("'numpy.ndarray' object has no attribute 'iloc'")
Traceback (most recent call last):
  File "C:\Users\juminji\AppData\Roaming\Python\Python39\site-packages\optuna\study\_optimize.py", line 213, in _run_trial
    value_or_values = func(trial)
  File "C:\Users\juminji\AppData\Local\Temp\ipykernel_18164\1359528307.py", line 18, in objective
    rmse = lg_nrmse(y_test, lgb_model.predict(X_test))
  File "C:\Users\juminji\AppData\Local\Temp\ipykernel_18164\3385352744.py", line 8, in lg_nrmse
    rmse = metrics.mean_squared_error(gt.iloc[:,idx], preds.iloc[:,idx], squared=False)
AttributeError: 'numpy.ndarray' object has no attribute 'iloc'


AttributeError: 'numpy.ndarray' object has no attribute 'iloc'

---

In [151]:
cat_reg = MultiOutputRegressor(cat_reg).fit(X_train,y_train)
y_cat=pd.DataFrame(cat_reg.predict(X_test))
score_cat=lg_nrmse(y_test, y_cat)
print(score_cat)

1.9691209877250837


---

In [157]:
lgb=LGBMRegressor(random_state = 42, learning_rate=0.05, n_estimators=200,
                 object='multiclass', metric='rmse')
cat_reg=CatBoostRegressor(depth = 4, random_state = 42, loss_function = 'RMSE', n_estimators = 3000, learning_rate = 0.03, verbose = 0, task_type='GPU')

lgb = MultiOutputRegressor(lgb).fit(train,target)
cat_reg = MultiOutputRegressor(cat_reg).fit(train,target)

pred_lgb = lgb.predict(test.iloc[:,1:])
#pred_cat = cat_reg.predict(test.iloc[:,1:])

CatBoostError: C:/Program Files (x86)/Go Agent/pipelines/BuildMaster/catboost.git/catboost/libs/data/model_dataset_compatibility.cpp:81: At position 0 should be feature with name PCB 체결 시 단계별 누름량(Step 1) (found X_01).

In [164]:
test_cat = test.iloc[:,1:].copy()
test_cat.columns=x_info.설명
test_cat

설명,PCB 체결 시 단계별 누름량(Step 1),PCB 체결 시 단계별 누름량(Step 2),방열 재료 1 무게,1차 검사 통과 여부,PCB 체결 시 단계별 누름량(Step 3),PCB 체결 시 단계별 누름량(Step 4),방열 재료 1 면적,방열 재료 2 면적,방열 재료 3 면적,방열 재료 2 무게,...,3차 검사 통과 여부,4차 검사 통과 여부,Cal 투입 전 대기 시간,RF1 부분 SMT 납 량,RF2 부분 SMT 납 량,RF3 부분 SMT 납 량,RF4 부분 SMT 납 량,RF5 부분 SMT 납 량,RF6 부분 SMT 납 량,RF7 부분 SMT 납 량
0,68.504,103.321,76.67,1,101.867,73.963,30.51,63.57,239.80,0.0,...,1,1,17227.63,138.130429,129.460682,141.506570,133.427229,129.711498,133.138096,121.859684
1,67.485,103.320,69.37,1,101.992,67.845,28.03,116.99,189.23,0.0,...,1,1,17134.53,136.148839,128.266277,145.911745,131.196417,132.411480,133.629025,124.178623
2,69.524,103.320,68.97,1,101.884,77.022,29.65,205.68,214.93,0.0,...,1,1,14860.83,120.447446,119.988804,132.099908,120.450155,130.051708,128.252972,114.475628
3,69.524,103.320,65.87,1,101.866,73.963,28.15,103.38,180.80,0.0,...,1,1,15252.53,133.994695,125.069180,147.507669,123.142653,125.963665,139.666592,126.589253
4,73.603,103.321,66.67,1,101.891,74.983,29.92,71.20,231.93,0.0,...,1,1,10752.23,137.918202,135.116192,138.600473,127.173033,137.252712,134.411335,124.020016
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
39603,68.504,103.320,63.97,1,103.157,68.864,29.49,116.35,284.16,0.0,...,1,1,62123.53,127.741246,126.494312,139.119905,125.271109,128.284572,140.176945,128.292843
39604,68.504,103.320,61.37,1,103.137,68.864,32.29,116.28,272.41,0.0,...,1,1,61844.13,127.767377,124.062809,138.238664,119.879393,127.322529,137.312047,131.570614
39605,69.524,103.320,63.67,1,103.149,69.884,30.00,113.05,295.54,0.0,...,1,1,60277.53,128.593640,124.774037,138.659624,123.999571,126.075542,135.656132,127.671108
39606,67.485,103.321,61.77,1,103.148,67.845,32.05,115.05,267.26,0.0,...,1,1,60236.73,121.110646,125.471699,134.989984,120.889578,129.296909,132.673977,131.882893


In [166]:
pred_cat = cat_reg.predict(test_cat)

In [167]:
ss = pd.read_csv('sample_submission.csv')
sub_col = ss.columns

In [168]:
testid = pd.DataFrame(test.ID)
sub_lgb = pd.concat([testid,pd.DataFrame(pred_lgb)],axis=1)
sub_lgb.columns=sub_col

sub_cat = pd.concat([testid,pd.DataFrame(pred_cat)],axis=1)
sub_cat.columns=sub_col

In [169]:
sub_lgb

,ID,Y_01,Y_02,Y_03,Y_04,Y_05,Y_06,Y_07,Y_08,Y_09,Y_10,Y_11,Y_12,Y_13,Y_14
0,TEST_00001,1.398496,1.264836,1.089157,13.873701,31.678043,16.652168,3.147093,-26.170992,-26.066434,-22.171733,24.443781,-26.067760,-26.008347,-26.063450
1,TEST_00002,1.502697,1.202858,1.162804,13.892128,30.990164,16.621858,3.149449,-26.187068,-26.164701,-22.289808,24.389927,-26.144824,-26.110643,-26.104030
2,TEST_00003,1.454201,1.195221,1.068573,15.627988,32.229865,16.568398,3.115233,-25.977655,-26.079156,-22.146952,24.557579,-25.831422,-25.917776,-26.091815
3,TEST_00004,1.455969,1.111128,1.044659,15.137877,32.542980,16.999286,3.119903,-25.656610,-25.724429,-21.879496,24.842271,-25.649061,-25.575494,-25.601500
4,TEST_00005,1.260136,1.008171,0.986302,14.792987,31.941781,17.112870,3.118361,-25.746057,-25.677640,-21.843961,24.834058,-25.637030,-25.579100,-25.642129
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
39603,TEST_39604,1.223029,0.938046,0.981755,12.599206,31.003259,16.712040,3.178530,-26.511637,-26.458943,-22.945233,24.360101,-26.404092,-26.443752,-26.500434
39604,TEST_39605,1.229804,0.875320,0.961995,13.940219,31.576347,16.744176,3.143903,-26.387739,-26.324008,-22.850807,24.564357,-26.193492,-26.389327,-26.252994
39605,TEST_39606,1.178628,0.954463,1.023871,13.092866,31.417220,16.633232,3.170174,-26.493993,-26.515544,-22.851285,24.262313,-26.434580,-26.455363,-26.481135
39606,TEST_39607,1.229573,0.847144,0.878465,13.298447,31.537479,16.651694,3.170349,-26.497312,-26.465595,-22.696987,24.433562,-26.459552,-26.450348,-26.399943


In [170]:
sub_cat

,ID,Y_01,Y_02,Y_03,Y_04,Y_05,Y_06,Y_07,Y_08,Y_09,Y_10,Y_11,Y_12,Y_13,Y_14
0,TEST_00001,1.421961,1.215037,1.106688,12.823180,31.145234,17.046575,3.344131,-26.116353,-26.128538,-22.221823,24.397200,-26.047201,-26.000171,-26.055989
1,TEST_00002,1.516781,1.220399,1.146119,13.618164,31.062698,16.644358,3.157988,-26.147962,-26.191816,-22.217999,24.298711,-26.098550,-26.131692,-26.136326
2,TEST_00003,1.435620,1.103161,1.025109,14.724641,32.034251,15.553407,2.988623,-25.949128,-25.895170,-22.066171,24.693474,-25.850805,-25.811387,-25.854753
3,TEST_00004,1.394833,1.142935,1.027319,15.246657,32.754324,17.102868,3.119232,-25.644346,-25.615742,-21.783453,24.993671,-25.625453,-25.547835,-25.578307
4,TEST_00005,1.361973,1.026207,0.961435,14.700999,31.729712,16.979303,3.108695,-25.659844,-25.687383,-21.897356,24.681610,-25.591042,-25.596698,-25.617251
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
39603,TEST_39604,1.277249,0.996298,0.991755,13.203207,31.162530,16.717701,3.151287,-26.418567,-26.413608,-22.750135,24.505135,-26.332783,-26.335925,-26.363446
39604,TEST_39605,1.225347,0.875311,1.001876,14.160517,31.616164,16.698317,3.210761,-26.394527,-26.344953,-22.756355,24.639727,-26.297772,-26.305929,-26.302032
39605,TEST_39606,1.253040,0.912348,0.970135,12.902595,31.124904,16.618580,3.156887,-26.434110,-26.431685,-22.710497,24.380073,-26.372794,-26.371092,-26.401182
39606,TEST_39607,1.185509,0.834565,0.961420,13.185367,31.329442,16.615217,3.202745,-26.501664,-26.482921,-22.778643,24.401894,-26.440642,-26.439435,-26.448679


In [176]:
pred = pred_lgb*0.6 + pred_cat*0.4
sub = pd.concat([testid,pd.DataFrame(pred)],axis=1)
sub.columns=sub_col
sub

,ID,Y_01,Y_02,Y_03,Y_04,Y_05,Y_06,Y_07,Y_08,Y_09,Y_10,Y_11,Y_12,Y_13,Y_14
0,TEST_00001,1.407882,1.244916,1.096169,13.453493,31.464919,16.809931,3.225908,-26.149136,-26.091275,-22.191769,24.425149,-26.059536,-26.005077,-26.060466
1,TEST_00002,1.508331,1.209874,1.156130,13.782543,31.019178,16.630858,3.152865,-26.171426,-26.175547,-22.261085,24.353441,-26.126314,-26.119063,-26.116949
2,TEST_00003,1.446768,1.158397,1.051187,15.266649,32.151619,16.162402,3.064589,-25.966244,-26.005562,-22.114639,24.611937,-25.839175,-25.875221,-25.996990
3,TEST_00004,1.431515,1.123851,1.037723,15.181389,32.627518,17.040719,3.119635,-25.651704,-25.680954,-21.841079,24.902831,-25.639618,-25.564431,-25.592222
4,TEST_00005,1.300871,1.015386,0.976355,14.756192,31.856954,17.059443,3.114495,-25.711572,-25.681537,-21.865319,24.773079,-25.618634,-25.586139,-25.632178
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
39603,TEST_39604,1.244717,0.961347,0.985755,12.840807,31.066967,16.714304,3.167633,-26.474409,-26.440809,-22.867194,24.418115,-26.375568,-26.400621,-26.445639
39604,TEST_39605,1.228021,0.875316,0.977948,14.028338,31.592274,16.725833,3.170646,-26.390454,-26.332386,-22.813026,24.594505,-26.235204,-26.355968,-26.272609
39605,TEST_39606,1.208393,0.937617,1.002377,13.016758,31.300294,16.627371,3.164859,-26.470039,-26.482000,-22.794970,24.309417,-26.409866,-26.421655,-26.449154
39606,TEST_39607,1.211948,0.842113,0.911647,13.253215,31.454264,16.637103,3.183308,-26.499053,-26.472526,-22.729649,24.420894,-26.451988,-26.445983,-26.419437


In [178]:
sub.to_csv('sub_lgb_cat_64_0819.csv',index=False)
pd.read_csv('sub_lgb_cat_64_0819.csv')

,ID,Y_01,Y_02,Y_03,Y_04,Y_05,Y_06,Y_07,Y_08,Y_09,Y_10,Y_11,Y_12,Y_13,Y_14
0,TEST_00001,1.407882,1.244916,1.096169,13.453493,31.464919,16.809931,3.225908,-26.149136,-26.091275,-22.191769,24.425149,-26.059536,-26.005077,-26.060466
1,TEST_00002,1.508331,1.209874,1.156130,13.782543,31.019178,16.630858,3.152865,-26.171426,-26.175547,-22.261085,24.353441,-26.126314,-26.119063,-26.116949
2,TEST_00003,1.446768,1.158397,1.051187,15.266649,32.151619,16.162402,3.064589,-25.966244,-26.005562,-22.114639,24.611937,-25.839175,-25.875221,-25.996990
3,TEST_00004,1.431515,1.123851,1.037723,15.181389,32.627518,17.040719,3.119635,-25.651704,-25.680954,-21.841079,24.902831,-25.639618,-25.564431,-25.592222
4,TEST_00005,1.300871,1.015386,0.976355,14.756192,31.856954,17.059443,3.114495,-25.711572,-25.681537,-21.865319,24.773079,-25.618634,-25.586139,-25.632178
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
39603,TEST_39604,1.244717,0.961347,0.985755,12.840807,31.066967,16.714304,3.167633,-26.474409,-26.440809,-22.867194,24.418115,-26.375568,-26.400621,-26.445639
39604,TEST_39605,1.228021,0.875316,0.977948,14.028338,31.592274,16.725833,3.170646,-26.390454,-26.332386,-22.813026,24.594505,-26.235204,-26.355968,-26.272609
39605,TEST_39606,1.208393,0.937617,1.002377,13.016758,31.300294,16.627371,3.164859,-26.470039,-26.482000,-22.794970,24.309417,-26.409866,-26.421655,-26.449154
39606,TEST_39607,1.211948,0.842113,0.911647,13.253215,31.454264,16.637103,3.183308,-26.499053,-26.472526,-22.729649,24.420894,-26.451988,-26.445983,-26.419437
